In [1]:
#import libraries

import pandas as pd
import numpy as np
import yfinance as yf
import statistics as stat

In [2]:
#need balance sheet
#need income statement
#need cashflow statement

#storage
balance = []
income = []
cashflow = []
q = []

#scores and metrics
profit_score = 0
leverage_score = 0

In [221]:
#obtain data from standard stock
def get_data(ticker):
    stock = yf.Ticker(ticker)
    balance = stock.quarterly_balance_sheet
    income = stock.quarterly_income_stmt
    cashflow = stock.quarterly_cashflow
    q = balance.columns

    return cashflow

In [223]:
#obtain profitability
def profit(ticker):
    
    stock = yf.Ticker(ticker)
    balance = stock.quarterly_balance_sheet
    income = stock.quarterly_income_stmt
    cashflow = stock.quarterly_cashflow
    q = balance.columns
    
    profit_score = 0
    
    ############################################################################
    #Data

    NI = 'Net Income'
    TA = 'Total Assets'
    OC = 'Net Income From Continuing Operations'

    if OC not in cashflow[q[0]]:
        OC = 'Operating Cash Flow'
    if OC not in cashflow[q[0]]:
        OC = 'Classesof Cash Receiptsfrom Operating Activities'
    
    #Net Income
    net_income1 = income[q[0]][NI]
    net_income2 = income[q[1]][NI]
    net_income3 = income[q[2]][NI]
    net_income4 = income[q[3]][NI]
    #Total Asset
    Tasset1 = balance[q[0]][TA]
    Tasset2 = balance[q[1]][TA]
    Tasset3 = balance[q[2]][TA]
    Tasset4 = balance[q[3]][TA]
    #Operating Cashflow
    OCF1 = cashflow[q[0]][OC]
    OCF2 = cashflow[q[1]][OC]    
    OCF3 = cashflow[q[2]][OC]
    OCF4 = cashflow[q[3]][OC]

    
    #############################################################################
    #Formulas
    
    #ROA
    ROA1 = net_income1 / stat.mean([Tasset1,Tasset2])
    ROA2 = net_income2 / stat.mean([Tasset2,Tasset3])
    ROA3 = net_income3 / stat.mean([Tasset3,Tasset4])
    ROA4 = net_income4 / Tasset4

    #############################################################################
    #Scores
    
    #ROA Score if positive
    ROA_score1a = 1 if ROA1 > 0 else 0
    ROA_score2a = 1 if ROA2 > 0 else 0
    ROA_score3a = 1 if ROA3 > 0 else 0
    ROA_score4a = 1 if ROA4 > 0 else 0
    finalscore1 = stat.mean([ROA_score1a,ROA_score2a,ROA_score3a,ROA_score4a])
    
    #ROA Score if > than previous
    ROA_score1b = 1 if ROA1 > ROA2 else 0
    ROA_score2b = 1 if ROA2 > ROA3 else 0
    ROA_score3b = 1 if ROA3 > ROA4 else 0
    finalscore2 = stat.mean([ROA_score1b,ROA_score2b,ROA_score3b])

    #Operating Cashflow Score if positive
    OCF_score1 = 1 if OCF1 > 0 else 0
    OCF_score2 = 1 if OCF2 > 0 else 0
    OCF_score3 = 1 if OCF3 > 0 else 0
    OCF_score4 = 1 if OCF4 > 0 else 0
    finalscore3 = stat.mean([OCF_score1,OCF_score2,OCF_score3,OCF_score4])

    #Accruals Score if OCF/Tasset is higher than ROA
    Acc_score1 = 1 if OCF1/Tasset1 > ROA1 else 0
    Acc_score2 = 1 if OCF2/Tasset2 > ROA2 else 0
    Acc_score3 = 1 if OCF3/Tasset3 > ROA3 else 0
    Acc_score4 = 1 if OCF4/Tasset4 > ROA4 else 0
    finalscore4 = stat.mean([Acc_score1,Acc_score2,Acc_score3,Acc_score4])

    ##############################################################################
    #Sum all scores

    profit_score = round((finalscore1 + finalscore2 + finalscore3 + finalscore4)/4 * 100, 2)
    return profit_score

In [5]:
def leverage(ticker):
    
    stock = yf.Ticker(ticker)
    balance = stock.quarterly_balance_sheet
    income = stock.quarterly_income_stmt
    cashflow = stock.quarterly_cashflow
    q = balance.columns
    
    leverage_score = 0

    ###############################################################################
    #Check Long-term Debt
    
    debt_score = 0
    
    try:
        lt_debt = balance[q[0]]['Long Term Debt']
        total_assets = stat.mean([balance[q[0]]['Total Assets'],balance[q[1]]['Total Assets']])
        debt_ratio = lt_debt / total_assets
        if debt_ratio < 1:
            debt_score = 1 - debt_ratio
        else:
            debt_score = 1 - 1
    except:
        debt_score = 0.5

    ###############################################################################
    #Check Current Ratio
    current_score = 0

    try:
        c_assets = balance[q[0]]['Current Assets']
        c_liab  = balance[q[0]]['Current Liabilities']
        current_ratio = c_assets / c_liab
        if current_ratio > 0 and current_ratio <= 1:
            current_score = current_ratio
        elif currrent_ratio > 1:
            current_score = 1
        else:
            current_score = 0
    except:
        current_score = 0.5
        
    ################################################################################
    #Check Number of Shares
    share_score = 1
    
    shares1 = balance[q[0]]['Share Issued']
    shares2 = stat.mean([balance[q[1]]['Share Issued'], balance[q[2]]['Share Issued']])

    if shares1 > shares2:
        share_score = 0

    ################################################################################
    #Sum all scores
    
    leverage_score = round((debt_score + current_score + share_score)/3 * 100, 2)
    
    return leverage_score

In [64]:
def eff(ticker):
    
    stock = yf.Ticker(ticker)
    balance = stock.quarterly_balance_sheet
    income = stock.quarterly_income_stmt
    cashflow = stock.quarterly_cashflow
    q = balance.columns
    eff_score = 0

    ################################################################################
    #Check Gross Margin

    gm_score = 0

    try:
        rev1 = income[q[0]]['Total Revenue']
        rev2 = income[q[1]]['Total Revenue']
        rev3 = income[q[2]]['Total Revenue']
        COGS1 = income[q[0]]['Cost Of Revenue']
        COGS2 = income[q[1]]['Cost Of Revenue']
        COGS3 = income[q[2]]['Cost Of Revenue']
        gm1 = (rev1 - COGS1)/(rev1)
        gm2 = stat.mean([(rev2 - COGS2)/(rev2),(rev3 - COGS3)/(rev3)])

        if gm1 > 0.1:
            gm_score = 0.25
            if gm1 > gm2:
                gm_score += round((((gm1/gm2) - 1)*10)/0.25, 2)
                if gm_score > 1:
                    gm_score = 1
        else:
            gm_score = 0

    except:
        gm_score = 0.5

    ##################################################################################
    #Check Asset Turnover

    at_score = 0
    TA = 'Total Assets'
    
    try:
        rev1 = income[q[0]]['Total Revenue']
        rev2 = income[q[1]]['Total Revenue']
        Tasset1 = balance[q[0]][TA]
        Tasset2 = balance[q[1]][TA]
        Tasset3 = balance[q[2]][TA]
        ast1 = stat.mean([Tasset1, Tasset2])
        ast2 = stat.mean([Tasset2, Tasset3])
        AT1 = rev1/ast1
        AT2 = rev2/ast2

        if AT1 > 0.1:
            at_score = 0.25
            if AT1/AT2 >= 1:
                at_score += round((((AT1/AT2) - 1))/0.25, 2)
                if at_score > 1:
                    at_score = 1
        else:
            at_score = 0
    except:
        at_score = 0.5
        
    ######################################################################################
    #Sum

    eff_score = round(((gm_score + at_score)/2)*100, 2)

    return eff_score

In [68]:
def analysis(tickers):
    
    Profit_score = []
    Leverage_score = []
    Eff_score = []
    
    for ticker in tickers:
        try:
            Profit_score.append(profit(ticker))
        except:
            Profit_score.append('')
        try:
            Leverage_score.append(leverage(ticker))
        except:
            Leverage_score.append('')
        try:
            Eff_score.append(eff(ticker))
        except:
            Eff_score.append('')
    
    stocks_df = {'Companies': tickers}
    stocks_df['Profit Score'] = Profit_score
    stocks_df['Leverage Score'] = Leverage_score
    stocks_df['Efficiency Score'] = Eff_score
    stocks_df['Profit Score'] = pd.to_numeric(stocks_df['Profit Score'])
    stocks_df['Leverage Score'] = pd.to_numeric(stocks_df['Leverage Score'])
    stocks_df['Efficiency Score'] = pd.to_numeric(stocks_df['Efficiency Score']) 
    final_df = pd.DataFrame(stocks_df).set_index('Companies').dropna()

    return final_df

In [213]:
Defense = ['LMT','RTX','NOC','GD','LHX','HON','TXT','SWBI',
            '7011.T','HXL','KTOS','BA','OSIS']
Software = ['PLTR','SNAP','MSFT','TOST','ORCL','CRM','SNOW',
            'S','CRWD','ADBE','SPLK','TTWO','SAP','BB','MBLY'
            '0700.HK','9999.HK','GOOG','AMZN','U','RBLX']
Machine = ['TEX','CAT','DE','PCAR','CMI','AGCO','ALG','CNHI','BWA']
Snack = ['MDLZ','PEP','HSY','POST','KO','GIS','COKE','KHC','CPB']
Insurance = ['MFC.TO','SLF.TO','PRU','MET','AIG','RGA','BNRE.TO','POW.TO']
Banking = ['JPM','BAC','WFC','RY.TO','BMO.TO','TD.TO','C','MUFG','SMFG',
           'BNS.TO','TCBI', 'BK','0939.HK','0011.HK','3328.HK','3988.HK']
Resturants = ['TXRH','MCD','YUM','SBUX','DPZ','CMG','QSR.TO','WING','WEN','SHAK','PZZA']
Electronics = ['AAPL','1810.HK','SONY','DELL','HPQ','HPE']
Resources = ['CQP','LNG','5411.T','5401.T','5406','NEE','ATO','NI','NFE','0386.HK','XOM','CVX','SHEL','NP','IMO']
Health = ['JNJ','GSK','ABBV','MRNA','BMY','NVO','MRK','AZN','AMGN']
Semi = ['NVDA','AMD','NXPI','TSMC','ASML','INTC','MU','TXN','ADI','ARM','MRVL','SWKS']
Motor = ['TSLA','TM','RACE','STLA','HMC','GM','F','1211.HK','NIO','0175.HK']

In [214]:
software_df = analysis(Software)

In [242]:
df = software_df
df['Total Score'] = round(df['Profit Score'] * 0.60 + df['Leverage Score'] * 0.10 + df['Efficiency Score'] * 0.30,2)
df = df.drop(df[df['Total Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Efficiency Score'] < 45].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Leverage Score'] < 30].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Profit Score'] < 60].index).sort_values('Total Score', ascending = False)
df

,Profit Score,Leverage Score,Efficiency Score,Total Score
Companies,,,,
9999.HK,72.92,83.25,94.0,80.28
AMZN,75.00,61.93,70.5,72.34
MSFT,75.00,80.07,62.5,71.76
PLTR,85.42,33.33,48.5,69.14
SAP,64.58,50.00,75.5,66.40


In [86]:
defense_df = analysis(Defense)

In [100]:
df = defense_df
df['Total Score'] = round(df['Profit Score'] * 0.60 + df['Leverage Score'] * 0.10 + df['Efficiency Score'] * 0.30,2)
df = df.drop(df[df['Total Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Efficiency Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Leverage Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Profit Score'] < 50].index).sort_values('Total Score', ascending = False)
df

,Profit Score,Leverage Score,Efficiency Score,Total Score
Companies,,,,
LHX,79.17,53.25,62.5,71.58
LMT,70.83,72.77,58.0,67.18
GD,66.67,77.52,63.5,66.80
TXT,66.67,76.92,58.5,65.24


In [102]:
machine_df = analysis(Machine)

In [107]:
df = machine_df
df['Total Score'] = round(df['Profit Score'] * 0.60 + df['Leverage Score'] * 0.10 + df['Efficiency Score'] * 0.30,2)
df = df.drop(df[df['Total Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Efficiency Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Leverage Score'] < 40].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Profit Score'] < 30].index).sort_values('Total Score', ascending = False)
df

,Profit Score,Leverage Score,Efficiency Score,Total Score
Companies,,,,
CAT,66.67,73.94,57.5,64.65
AGCO,64.58,44.37,62.5,61.93


In [109]:
snack_df = analysis(Snack)

In [251]:
df = snack_df
df['Total Score'] = round(df['Profit Score'] * 0.60 + df['Leverage Score'] * 0.10 + df['Efficiency Score'] * 0.30,2)
df = df.drop(df[df['Total Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Efficiency Score'] < 45].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Leverage Score'] < 45].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Profit Score'] < 60].index).sort_values('Total Score', ascending = False)
df

,Profit Score,Leverage Score,Efficiency Score,Total Score
Companies,,,,
CPB,72.92,86.64,91.5,79.87
KO,79.17,71.71,62.5,73.42
MDLZ,85.42,45.95,49.5,70.70
HSY,66.67,71.92,60.0,65.19


In [116]:
insurance_df = analysis(Insurance)

In [253]:
df = insurance_df
df['Total Score'] = round(df['Profit Score'] * 0.60 + df['Leverage Score'] * 0.10 + df['Efficiency Score'] * 0.30,2)
df = df.drop(df[df['Total Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Efficiency Score'] < 20].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Leverage Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Profit Score'] < 70].index).sort_values('Total Score', ascending = False)
df

,Profit Score,Leverage Score,Efficiency Score,Total Score
Companies,,,,
POW.TO,91.67,82.39,25.0,70.74
RGA,91.67,81.65,25.0,70.67
MFC.TO,83.33,82.85,25.0,65.78
SLF.TO,83.33,82.34,25.0,65.73
AIG,72.92,81.88,25.0,59.44


In [130]:
banking_df = analysis(Banking)

In [254]:
df = banking_df
df['Total Score'] = round(df['Profit Score'] * 0.60 + df['Leverage Score'] * 0.10 + df['Efficiency Score'] * 0.30,2)
df = df.drop(df[df['Total Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Efficiency Score'] < 20].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Leverage Score'] < 40].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Profit Score'] < 70].index).sort_values('Total Score', ascending = False)
df

,Profit Score,Leverage Score,Efficiency Score,Total Score
Companies,,,,
BNS.TO,91.67,45.47,25.0,67.05
C,85.42,46.16,25.0,63.37
3988.HK,79.17,66.67,25.0,61.67
BMO.TO,81.25,46.41,25.0,60.89
RY.TO,81.25,44.56,25.0,60.71
BAC,72.92,80.25,25.0,59.28
TD.TO,70.83,80.26,25.0,58.02
0939.HK,72.92,66.67,25.0,57.92
BK,72.92,47.61,25.0,56.01


In [139]:
resturants_df = analysis(Resturants)

In [262]:
df = resturants_df
df['Total Score'] = round(df['Profit Score'] * 0.60 + df['Leverage Score'] * 0.10 + df['Efficiency Score'] * 0.30,2)
df = df.drop(df[df['Total Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Efficiency Score'] < 60].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Leverage Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Profit Score'] < 50].index).sort_values('Total Score', ascending = False)
df

,Profit Score,Leverage Score,Efficiency Score,Total Score
Companies,,,,
WING,79.17,50.00,83.5,77.55
SBUX,75.00,77.19,64.0,71.92
WEN,77.08,66.00,62.5,71.60


In [162]:
electronics_df = analysis(Electronics)

In [163]:
df = electronics_df
df['Total Score'] = round(df['Profit Score'] * 0.60 + df['Leverage Score'] * 0.10 + df['Efficiency Score'] * 0.30,2)
df = df.drop(df[df['Total Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Efficiency Score'] < 60].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Leverage Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Profit Score'] < 60].index).sort_values('Total Score', ascending = False)
df

,Profit Score,Leverage Score,Efficiency Score,Total Score
Companies,,,,
1810.HK,85.42,80.77,62.5,78.08
AAPL,64.58,90.36,72.5,69.53


In [227]:
resources_df = analysis(Resources)

In [228]:
df = resources_df
df['Total Score'] = round(df['Profit Score'] * 0.60 + df['Leverage Score'] * 0.10 + df['Efficiency Score'] * 0.30,2)
df = df.drop(df[df['Total Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Efficiency Score'] < 60].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Leverage Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Profit Score'] < 60].index).sort_values('Total Score', ascending = False)
df

,Profit Score,Leverage Score,Efficiency Score,Total Score
Companies,,,,
0386.HK,91.67,93.40,77.0,87.44
SHEL,83.33,50.00,71.0,76.30
IMO,64.58,80.65,95.0,75.31
5401.T,77.08,66.67,62.5,71.66


In [184]:
health_df = analysis(Health)

In [194]:
df = health_df
df['Total Score'] = round(df['Profit Score'] * 0.60 + df['Leverage Score'] * 0.10 + df['Efficiency Score'] * 0.30,2)
df = df.drop(df[df['Total Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Efficiency Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Leverage Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Profit Score'] < 70].index).sort_values('Total Score', ascending = False)
df

,Profit Score,Leverage Score,Efficiency Score,Total Score
Companies,,,,
BMY,83.33,71.74,58.5,74.72
JNJ,72.92,78.48,62.5,70.35


In [196]:
semi_df = analysis(Semi)

In [209]:
df = semi_df
df['Total Score'] = round(df['Profit Score'] * 0.60 + df['Leverage Score'] * 0.10 + df['Efficiency Score'] * 0.30,2)
df = df.drop(df[df['Total Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Efficiency Score'] < 40].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Leverage Score'] < 30].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Profit Score'] < 50].index).sort_values('Total Score', ascending = False)
df

,Profit Score,Leverage Score,Efficiency Score,Total Score
Companies,,,,
NVDA,75.00,77.90,100.0,82.79
NXPI,91.67,35.81,44.5,71.93
SWKS,77.08,46.08,58.0,68.26
ASML,64.58,45.95,60.5,61.49
AMD,66.67,49.16,50.0,59.92
INTC,52.08,41.71,50.0,50.42


In [225]:
motor_df = analysis(Motor)

In [226]:
df = motor_df
df['Total Score'] = round(df['Profit Score'] * 0.60 + df['Leverage Score'] * 0.10 + df['Efficiency Score'] * 0.30,2)
df = df.drop(df[df['Total Score'] < 50].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Efficiency Score'] < 60].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Leverage Score'] < 60].index).sort_values('Total Score', ascending = False)
df = df.drop(df[df['Profit Score'] < 30].index).sort_values('Total Score', ascending = False)
df

,Profit Score,Leverage Score,Efficiency Score,Total Score
Companies,,,,
1211.HK,91.67,88.03,79.5,87.66
HMC,77.08,76.86,66.0,73.73
TM,70.83,75.47,67.0,70.14


In [238]:
analysis(['0700.HK'])

,Profit Score,Leverage Score,Efficiency Score
Companies,,,
0700.HK,85.42,77.04,68.0
